In [ ]:
!pip install torcheval --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 5.4 MB/s eta 0:00:00


In [ ]:
import torch
import torchvision
import pandas
import cv2 as cv
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torcheval.metrics import BinaryF1Score, BinaryRecall, BinaryPrecision, BinaryAccuracy
from tqdm.auto import tqdm
import time

In [ ]:
model_resnet = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 80.4MB/s]


In [ ]:
batch_size = 128
random_seed = 8
torch.manual_seed(random_seed)

In [ ]:
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
dataset = torchvision.datasets.ImageFolder(root="/content/drive/MyDrive/Sem_VII_project", transform=data_transforms)
print(dataset.classes)

['bleeding-not', 'bleeding-yes']


In [ ]:
train_len = int(0.7 * len(dataset))
val_len = int(0.1 * len(dataset))
test_len = len(dataset) - (train_len + val_len)

In [ ]:
train_ds, val_ds, test_ds = torch.utils.data.random_split(dataset, [train_len, val_len, test_len])

In [ ]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size = batch_size, shuffle=True)
valid_dl = torch.utils.data.DataLoader(val_ds, batch_size = batch_size)
test_dl = torch.utils.data.DataLoader(test_ds)

In [ ]:
# for param in model_resnet.parameters():
#  param.requires_grad = False
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(in_features=num_ftrs, out_features=1)

In [ ]:
if not torch.cuda.is_available():
  device = 'cpu'
else:
  device = 'cuda'
print(device)

cuda


In [ ]:
model_resnet.to(device)

In [ ]:
metric = BinaryF1Score(device=device)
prec = BinaryPrecision(device=device)
recall = BinaryRecall(device=device)
acc = BinaryAccuracy(device=device)

In [ ]:
def train(epochs, model):
  optimizer = torch.optim.Adadelta(model.parameters())
  loss_func = nn.BCEWithLogitsLoss(reduction='mean')
  for epoch in range(epochs):
    time.sleep(1)
    model.train()
    batch=0
    running_loss = 0
    for _, (inputs, labels) in enumerate(tqdm(train_dl)):
      inputs, labels = inputs.cuda(), labels.cuda()
      optimizer.zero_grad()
      with torch.set_grad_enabled(True):
        outputs = model(inputs)
        # preds = torch.round(outputs.sigmoid()).squeeze()
        loss = loss_func(outputs, labels.float().unsqueeze(1))
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"[+] epoch {epoch+1}/{epochs} running loss: {running_loss}")
    model.eval()
    with torch.inference_mode():
      valid_loss = 0
      total_acc = 0
      len_preds = 0
      for inputs, labels in valid_dl:
        inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model(inputs)
        preds = outputs.sigmoid().squeeze()
        loss = F.binary_cross_entropy(torch.sigmoid(outputs), labels.float().unsqueeze(1))
        valid_loss += loss.item()
        metric.update(preds, labels)
        prec.update(preds, labels)
        recall.update(preds, labels)
        acc.update(preds, labels)
      print(f"[-] total validation loss for epoch {epoch}: {valid_loss} accuracy: {acc.compute()} precision: {prec.compute()} recall: {recall.compute()} F1Score: {metric.compute()}")


In [ ]:
train(5, model_resnet)

  0%|          | 0/15 [00:00<?, ?it/s]

[+] epoch 1/5 running loss: 1.6496328716748394
[-] total validation loss for epoch 0: 1.587105412472738 accuracy: 0.8659003973007202 precision: 0.8023256063461304 recall: 0.9928057789802551 F1Score: 0.8874598145484924


  0%|          | 0/15 [00:00<?, ?it/s]

[+] epoch 2/5 running loss: 0.3670150625985116
[-] total validation loss for epoch 1: 0.07132333936169744 accuracy: 0.9233716726303101 precision: 0.8888888955116272 recall: 0.9784172773361206 F1Score: 0.9315068125724792


  0%|          | 0/15 [00:00<?, ?it/s]

[+] epoch 3/5 running loss: 0.03652457636781037
[-] total validation loss for epoch 2: 0.029540585819631815 accuracy: 0.946360170841217 precision: 0.9213483333587646 recall: 0.9832134246826172 F1Score: 0.9512761235237122


  0%|          | 0/15 [00:00<?, ?it/s]

[+] epoch 4/5 running loss: 0.014308024517958984
[-] total validation loss for epoch 3: 0.03196263680001721 accuracy: 0.9578543901443481 precision: 0.9398625493049622 recall: 0.9838129281997681 F1Score: 0.9613356590270996


  0%|          | 0/15 [00:00<?, ?it/s]

[+] epoch 5/5 running loss: 0.007560209807706997
[-] total validation loss for epoch 4: 0.032663044476066716 accuracy: 0.9647509455680847 precision: 0.9513213038444519 recall: 0.9841726422309875 F1Score: 0.9674682021141052


In [ ]:
def evaluate(model, test_data, eval_metric):
  model.eval()
  with torch.inference_mode():
    valid_loss = 0
    total_acc = 0
    len_preds = 0
    for inputs, labels in test_data:
      inputs = inputs.cuda()
      labels = labels.cuda()
      outputs = model(inputs)
      preds = torch.round(outputs.sigmoid()).squeeze(1)
      eval_metric.update(preds, labels)
      prec.update(preds, labels)
      recall.update(preds, labels)
      loss = F.binary_cross_entropy_with_logits(outputs, labels.float().unsqueeze(1))
      valid_loss += loss.item()
      total_acc += torch.tensor(torch.sum(preds == labels).item())
      len_preds += len(preds)
    print(f"[*] total binary cross entropy loss: {valid_loss} accuracy: {total_acc/len_preds} F1Score: {metric.compute()} BinaryRecall: {recall.compute()} Precision: {prec.compute()}")
    eval_metric.reset()
    prec.reset()
    recall.reset()


In [ ]:
metric.reset()
prec.reset()
recall.reset()
acc.reset()
evaluate(model_resnet, test_dl, metric)

[*] total binary cross entropy loss: 1.6024465096275051 accuracy: 1.0 F1Score: 1.0 BinaryRecall: 1.0 Precision: 1.0


In [ ]:
from PIL import Image
import os

In [ ]:
path = "/content/drive/MyDrive/Test Dataset 1"
d = {0.:0,
     1.:0}
transform = data_transforms
for i in os.listdir(path):
  img = Image.open(os.path.join(path, i))
  img = transform(img)
  img = img.to(device)
  img = torch.unsqueeze(img, dim=0)
  with torch.inference_mode():
    output = model_resnet(img).squeeze()
    pred = torch.round(output.sigmoid())
    print(i, pred, output.sigmoid())
    d[pred.item()] += 1


print(d[0.], d[1.])



A0009.png tensor(0., device='cuda:0') tensor(3.0542e-05, device='cuda:0')
A0008.png tensor(0., device='cuda:0') tensor(4.4112e-05, device='cuda:0')
A0001.png tensor(0., device='cuda:0') tensor(0.0302, device='cuda:0')
A0005.png tensor(0., device='cuda:0') tensor(0.0002, device='cuda:0')
A0012.png tensor(0., device='cuda:0') tensor(0.0001, device='cuda:0')
A0004.png tensor(0., device='cuda:0') tensor(3.9193e-05, device='cuda:0')
A0016.png tensor(0., device='cuda:0') tensor(2.9878e-07, device='cuda:0')
A0003.png tensor(0., device='cuda:0') tensor(0.0010, device='cuda:0')
A0000.png tensor(1., device='cuda:0') tensor(0.9990, device='cuda:0')
A0006.png tensor(0., device='cuda:0') tensor(0.0013, device='cuda:0')
A0002.png tensor(1., device='cuda:0') tensor(0.9992, device='cuda:0')
A0011.png tensor(0., device='cuda:0') tensor(0.0267, device='cuda:0')
A0014.png tensor(0., device='cuda:0') tensor(3.0277e-05, device='cuda:0')
A0013.png tensor(0., device='cuda:0') tensor(0.0007, device='cuda:0')
